In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud


In [3]:
job_data = pd.read_csv('./data/SF_jobs.csv')

In [4]:
job_data.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
#Functions for data cleaning
wnl = WordNetLemmatizer()

def text_cleaner(text):
    
    text = text.lower().split()
    text = [wnl.lemmatize(myWord) for myWord in text] # Lemmertization

    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    
    return text # return list of tokens

In [7]:
cleanDescription = []
for i in range(1500):
    cleanDescription.append(text_cleaner(job_data.job_description[i]))

In [8]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=10,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [ ]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [ ]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz